In [1]:
import pandas as pd
import numpy as np

import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Intro

In [2]:
# Load data
df = pd.read_csv('data/lead_scoring.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Prospect ID                                     9240 non-null   object 
 1   Lead Number                                     9240 non-null   int64  
 2   Lead Origin                                     9240 non-null   object 
 3   Lead Source                                     9204 non-null   object 
 4   Do Not Email                                    9240 non-null   object 
 5   Do Not Call                                     9240 non-null   object 
 6   Converted                                       9240 non-null   int64  
 7   TotalVisits                                     9103 non-null   float64
 8   Total Time Spent on Website                     9240 non-null   int64  
 9   Page Views Per Visit                     

## Handling Each Columns Values

In [4]:
## Untuk Kolom Prospect ID & Lead Number tidak ada masalah unique values dan non-null count sama yaitu `9240`

In [5]:
## untuk kolom Lead Origin memiliki value yang hanya 1 yaitu Quick Add Form, kita gabungkan dengan Lead Ads Form
print("\nBefore")
display(df['Lead Origin'].value_counts())

df['Lead Origin'] = np.where(df['Lead Origin'].str.contains("Add Form"), "Add Form", df['Lead Origin'])
print("\nAfter")
display(df['Lead Origin'].value_counts())


Before


Landing Page Submission    4886
API                        3580
Lead Add Form               718
Lead Import                  55
Quick Add Form                1
Name: Lead Origin, dtype: int64


After


Landing Page Submission    4886
API                        3580
Add Form                    719
Lead Import                  55
Name: Lead Origin, dtype: int64

In [6]:
## untuk kolom Lead Source, mari kita ganti Source yang kurang dari 1% populasi menjadi `Others`
# Tampilkan jumlah nilai untuk kolom 'Lead Source', dinormalisasi untuk menampilkan persentase
df['Lead Source'] = df['Lead Source'].fillna('Not Specified')
lead_source_percentage = df['Lead Source'].value_counts(normalize=True)
print("\nBefore")
display(lead_source_percentage)

# Lanjutkan dengan kode untuk mengganti Lead Source yang menyumbang kurang dari 1% populasi dengan 'Others'
threshold = 0.01  # 1% threshold
sources_to_replace = lead_source_percentage[lead_source_percentage < threshold].index

# Ganti Lead Source yang menyumbang kurang dari 1% populasi dengan 'Others'
df['Lead Source'] = df['Lead Source'].apply(lambda x: 'Others' if x in sources_to_replace else x)
print("\nAfter")
display(df['Lead Source'].value_counts(normalize=True))


Before


Google               0.310390
Direct Traffic       0.275216
Olark Chat           0.189935
Organic Search       0.124892
Reference            0.057792
Welingak Website     0.015368
Referral Sites       0.013528
Facebook             0.005952
Not Specified        0.003896
bing                 0.000649
google               0.000541
Click2call           0.000433
Press_Release        0.000216
Social Media         0.000216
Live Chat            0.000216
youtubechannel       0.000108
testone              0.000108
Pay per Click Ads    0.000108
welearnblog_Home     0.000108
WeLearn              0.000108
blog                 0.000108
NC_EDM               0.000108
Name: Lead Source, dtype: float64


After


Google              0.310390
Direct Traffic      0.275216
Olark Chat          0.189935
Organic Search      0.124892
Reference           0.057792
Welingak Website    0.015368
Referral Sites      0.013528
Others              0.012879
Name: Lead Source, dtype: float64

In [7]:
df['Do Not Email'].value_counts() ## tidak ada yang perlu dihandling

No     8506
Yes     734
Name: Do Not Email, dtype: int64

In [8]:
df['Do Not Call'].value_counts() ## tidak ada yang perlu dihandling

No     9238
Yes       2
Name: Do Not Call, dtype: int64

In [9]:
df['Converted'].value_counts() ## tidak ada yang perlu dihandling

0    5679
1    3561
Name: Converted, dtype: int64

In [10]:
display(df['TotalVisits'].describe()) ## ada null values, mari kita isi dengan `0`
df['TotalVisits'].fillna(0, inplace=True)
display(df['TotalVisits'].describe()) 

count    9103.000000
mean        3.445238
std         4.854853
min         0.000000
25%         1.000000
50%         3.000000
75%         5.000000
max       251.000000
Name: TotalVisits, dtype: float64

count    9240.000000
mean        3.394156
std         4.836682
min         0.000000
25%         0.000000
50%         3.000000
75%         5.000000
max       251.000000
Name: TotalVisits, dtype: float64

In [11]:
display(df['Page Views Per Visit'].describe()) ## ada null values, mari kita isi dengan `0`
df['Page Views Per Visit'].fillna(0, inplace=True)
display(df['Page Views Per Visit'].describe()) 

count    9103.000000
mean        2.362820
std         2.161418
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        55.000000
Name: Page Views Per Visit, dtype: float64

count    9240.000000
mean        2.327787
std         2.164258
min         0.000000
25%         0.000000
50%         2.000000
75%         3.000000
max        55.000000
Name: Page Views Per Visit, dtype: float64

In [12]:
## mari kita kelompokan aktivitas menjadi Good, Bad, dan Neutral
display(df['Last Activity'].value_counts())

# Definisikan kategori berdasarkan sudut pandang perusahaan (subjektif)
good_activities = ['Email Opened', 'Resubscribed to emails',  'Page Visited on Website', 'Form Submitted on Website', 'Email Link Clicked', 'Had a Phone Conversation', 'Approached upfront']
bad_activities = ['Email Bounced', 'Unreachable', 'Unsubscribed', 'Email Marked Spam']
neutral_activities = ['Email Received', 'SMS Sent', 'Olark Chat Conversation', 'Converted to Lead', 'View in browser link Clicked', 'Visited Booth in Tradeshow']

# Categorize 'Last Activity' menjadi Good, Bad, atau Neutral
df['Last Activity Category'] = df['Last Activity'].apply(lambda x: 'Good' if x in good_activities else ('Neutral' if x in neutral_activities else 'Bad')) # Jika `Last Activity` kosong, maka dianggap Bad

# Tampilkan
last_activity_category_counts = df['Last Activity Category'].value_counts()
display(last_activity_category_counts)

df = df.drop(columns=['Last Activity']) # kita coba drop saja

Email Opened                    3437
SMS Sent                        2745
Olark Chat Conversation          973
Page Visited on Website          640
Converted to Lead                428
Email Bounced                    326
Email Link Clicked               267
Form Submitted on Website        116
Unreachable                       93
Unsubscribed                      61
Had a Phone Conversation          30
Approached upfront                 9
View in browser link Clicked       6
Email Received                     2
Email Marked Spam                  2
Visited Booth in Tradeshow         1
Resubscribed to emails             1
Name: Last Activity, dtype: int64

Good       4500
Neutral    4155
Bad         585
Name: Last Activity Category, dtype: int64

In [13]:
# Mari kita ubah menjadi Indonesia & Outside Indoneisa
print("\nBefore")
display(df['Country'].value_counts())

df['Country'] = np.where(df['Country']=='Indonesia', 'Indonesia', 'Outside Indonesia') # termasuk yang Null/None

print("\nAfter")
display(df['Country'].value_counts())



Before


Indonesia               6492
United States             69
United Arab Emirates      53
Singapore                 24
Saudi Arabia              21
United Kingdom            15
Australia                 13
Qatar                     10
Hong Kong                  7
Bahrain                    7
Oman                       6
France                     6
unknown                    5
South Africa               4
Nigeria                    4
Germany                    4
Kuwait                     4
Canada                     4
Sweden                     3
China                      2
Asia/Pacific Region        2
Uganda                     2
Bangladesh                 2
Italy                      2
Belgium                    2
Netherlands                2
Ghana                      2
Philippines                2
Russia                     1
Switzerland                1
Vietnam                    1
Denmark                    1
Tanzania                   1
Liberia                    1
Malaysia      


After


Indonesia            6492
Outside Indonesia    2748
Name: Country, dtype: int64

In [14]:
# ubah `Select` menjadi 'Not Specified', begitu pula yang Null/None
print("\nBefore")
display(df['Specialization'].value_counts())

df['Specialization'] = np.where(df['Specialization']=='Select', 'Not Specified', df['Specialization'].fillna('Not Specified'))

print("\nAfter")
display(df['Specialization'].value_counts())


Before


Select                               1942
Finance Management                    976
Human Resource Management             848
Marketing Management                  838
Operations Management                 503
Business Administration               403
IT Projects Management                366
Supply Chain Management               349
Banking, Investment And Insurance     338
Travel and Tourism                    203
Media and Advertising                 203
International Business                178
Healthcare Management                 159
Hospitality Management                114
E-COMMERCE                            112
Retail Management                     100
Rural and Agribusiness                 73
E-Business                             57
Services Excellence                    40
Name: Specialization, dtype: int64


After


Not Specified                        3380
Finance Management                    976
Human Resource Management             848
Marketing Management                  838
Operations Management                 503
Business Administration               403
IT Projects Management                366
Supply Chain Management               349
Banking, Investment And Insurance     338
Travel and Tourism                    203
Media and Advertising                 203
International Business                178
Healthcare Management                 159
Hospitality Management                114
E-COMMERCE                            112
Retail Management                     100
Rural and Agribusiness                 73
E-Business                             57
Services Excellence                    40
Name: Specialization, dtype: int64

In [15]:
# ubah `Select` menjadi 'Not Specified', begitu pula yang Null/None
print("\nBefore")
display(df['How did you hear about Madugital'].value_counts())

df['How did you hear about Madugital'] = np.where(df['How did you hear about Madugital']=='Select', 'Not Specified', df['How did you hear about Madugital'].fillna('Not Specified'))

print("\nAfter")
display(df['How did you hear about Madugital'].value_counts())


Before


Select                   5043
Online Search             808
Word Of Mouth             348
Student of SomeSchool     310
Other                     186
Multiple Sources          152
Advertisements             70
Social Media               67
Email                      26
SMS                        23
Name: How did you hear about Madugital, dtype: int64


After


Not Specified            7250
Online Search             808
Word Of Mouth             348
Student of SomeSchool     310
Other                     186
Multiple Sources          152
Advertisements             70
Social Media               67
Email                      26
SMS                        23
Name: How did you hear about Madugital, dtype: int64

In [16]:
# isi missing values dengan "Not Specified"
df['What is your current occupation'] = df['What is your current occupation'].fillna('Not Specified')
df['What is your current occupation'].value_counts()

Unemployed              5600
Not Specified           2690
Working Professional     706
Student                  210
Other                     16
Housewife                 10
Businessman                8
Name: What is your current occupation, dtype: int64

In [17]:
# ubah Null/None menjadi 'Not Specified'
df['What matters most to you in choosing a product'] = df['What matters most to you in choosing a product'].fillna('Not Specified')
df['What matters most to you in choosing a product'].value_counts()

Healthy for life    6528
Not Specified       2709
Branding               2
Other                  1
Name: What matters most to you in choosing a product, dtype: int64

In [18]:
cols = ['Search', 'Magazine',
       'Newspaper Article', 'Madugital Telegram', 'Newspaper',
       'Digital Advertisement', 'Through Recommendations',
       'Receive More Updates About Our Products']
df[cols].value_counts() # kolom aman

Search  Magazine  Newspaper Article  Madugital Telegram  Newspaper  Digital Advertisement  Through Recommendations  Receive More Updates About Our Products
No      No        No                 No                  No         No                     No                       No                                         9217
Yes     No        No                 No                  No         No                     No                       No                                           11
No      No        No                 No                  No         No                     Yes                      No                                            4
                                                                    Yes                    No                       No                                            3
Yes     No        No                 No                  No         No                     Yes                      No                                            2
No      No        No    

In [19]:
## untuk kolom Tags, mari kita ganti yang kurang dari 1% populasi menjadi `Others`
# Tampilkan jumlah nilai untuk kolom 'Tags', dinormalisasi untuk menampilkan persentase
df['Tags'] = df['Tags'].fillna('Not Specified')
tags_percentage = df['Tags'].value_counts(normalize=True)
print("\nBefore")
display(tags_percentage)

# Lanjutkan dengan kode untuk mengganti Tags yang menyumbang kurang dari 1% populasi dengan 'Others'
threshold = 0.01  # 1% threshold
sources_to_replace = tags_percentage[tags_percentage < threshold].index

# Ganti Tags yang menyumbang kurang dari 1% populasi dengan 'Others'
df['Tags'] = df['Tags'].apply(lambda x: 'Others' if x in sources_to_replace else x)
print("\nAfter")
display(df['Tags'].value_counts(normalize=True))


Before


Not Specified                                        0.362879
Will revert after reading the email                  0.224242
Ringing                                              0.130195
Interested in other courses                          0.055519
Already a student                                    0.050325
Closed by Horizzon                                   0.038745
switched off                                         0.025974
Busy                                                 0.020130
Lost to EINS                                         0.018939
Not doing further education                          0.015693
Interested  in full time MBA                         0.012662
Graduation in progress                               0.012013
invalid number                                       0.008983
Diploma holder (Not Eligible)                        0.006818
wrong number given                                   0.005087
opp hangup                                           0.003571
number n


After


Not Specified                          0.362879
Will revert after reading the email    0.224242
Ringing                                0.130195
Interested in other courses            0.055519
Already a student                      0.050325
Closed by Horizzon                     0.038745
Others                                 0.032684
switched off                           0.025974
Busy                                   0.020130
Lost to EINS                           0.018939
Not doing further education            0.015693
Interested  in full time MBA           0.012662
Graduation in progress                 0.012013
Name: Tags, dtype: float64

In [20]:
# fill with Unknown
df['Lead Quality'] = df['Lead Quality'].fillna('Not Specified')

In [21]:
df[['Update me on Supply Chain Content', 'Get updates on DM Content']].value_counts() # aman

Update me on Supply Chain Content  Get updates on DM Content
No                                 No                           9240
dtype: int64

In [22]:
# ubah select jadi `Not Specified` dan yang null diisi sama
df['Lead Profile'] = np.where(df['Lead Profile']=='Select', 'Not Specified', df['Lead Profile'].fillna('Not Specified'))
df['Lead Profile'].value_counts()

Not Specified                  6855
Potential Lead                 1613
Other Leads                     487
Student of SomeSchool           241
Lateral Student                  24
Dual Specialization Student      20
Name: Lead Profile, dtype: int64

In [23]:
# ubah select jadi `Not Specified` dan yang null diisi sama
df['City'] = np.where(df['City']=='Select', 'Not Specified', df['City'].fillna('Not Specified'))
df['City'].value_counts()

Not Specified                    3669
Jakarta                          3222
Bodetabek                         752
Other Cities                      686
Cities in Banten & Jawa Barat     457
Other Metro Cities                380
Tier II Cities                     74
Name: City, dtype: int64

In [24]:
# isi missing values dengan 0 untuk Score / 00.Zero untku Index
df['Asymmetrique Activity Score'] = df['Asymmetrique Activity Score'].fillna(0)
df['Asymmetrique Activity Index'] = df['Asymmetrique Activity Index'].fillna('00.Zero')
display(df['Asymmetrique Activity Index'].value_counts())

df['Asymmetrique Profile Score'] = df['Asymmetrique Profile Score'].fillna(0)
df['Asymmetrique Profile Index'] = df['Asymmetrique Profile Index'].fillna('00.Zero')
display(df['Asymmetrique Profile Index'].value_counts())

00.Zero      4218
02.Medium    3839
01.High       821
03.Low        362
Name: Asymmetrique Activity Index, dtype: int64

00.Zero      4218
02.Medium    2788
01.High      2203
03.Low         31
Name: Asymmetrique Profile Index, dtype: int64

In [25]:
df[['I agree to pay the amount through cheque', 'A free copy of Mastering The Interview']].value_counts() # aman

I agree to pay the amount through cheque  A free copy of Mastering The Interview
No                                        No                                        6352
                                          Yes                                       2888
dtype: int64

In [26]:
## untuk kolom Last Notable Activity, mari kita ganti yang kurang dari 1% populasi menjadi `Others`
# Tampilkan jumlah nilai untuk kolom 'Last Notable Activity', dinormalisasi untuk menampilkan persentase
df['Last Notable Activity'] = df['Last Notable Activity'].fillna('Not Specified')
act_percentage = df['Last Notable Activity'].value_counts(normalize=True)
print("\nBefore")
display(act_percentage)

# Lanjutkan dengan kode untuk mengganti Last Notable Activity yang menyumbang kurang dari 1% populasi dengan 'Others'
threshold = 0.01  # 1% threshold
sources_to_replace = act_percentage[act_percentage < threshold].index

# Ganti Last Notable Activity yang menyumbang kurang dari 1% populasi dengan 'Others'
df['Last Notable Activity'] = df['Last Notable Activity'].apply(lambda x: 'Others' if x in sources_to_replace else x)
print("\nAfter")
display(df['Last Notable Activity'].value_counts(normalize=True))


Before


Modified                        0.368723
Email Opened                    0.305952
SMS Sent                        0.235065
Page Visited on Website         0.034416
Olark Chat Conversation         0.019805
Email Link Clicked              0.018723
Email Bounced                   0.006494
Unsubscribed                    0.005087
Unreachable                     0.003463
Had a Phone Conversation        0.001515
Email Marked Spam               0.000216
Approached upfront              0.000108
Resubscribed to emails          0.000108
View in browser link Clicked    0.000108
Form Submitted on Website       0.000108
Email Received                  0.000108
Name: Last Notable Activity, dtype: float64


After


Modified                   0.368723
Email Opened               0.305952
SMS Sent                   0.235065
Page Visited on Website    0.034416
Olark Chat Conversation    0.019805
Email Link Clicked         0.018723
Others                     0.017316
Name: Last Notable Activity, dtype: float64

## Persiapan Data

In [ ]:
# Menghapus duplikat
df = df.drop_duplicates()

cols = ['I agree to pay the amount through cheque', 'A free copy of Mastering The Interview', 'Update me on Supply Chain Content', 'Get updates on DM Content', 'Search', 'Magazine',
       'Newspaper Article', 'Madugital Telegram', 'Newspaper', 'Digital Advertisement', 'Through Recommendations', 'Receive More Updates About Our Products', 'Do Not Email', 'Do Not Call']

# Ganti 'Yes' dengan 1 dan 'No' dengan 0 pada kolom-kolom diatas
df[cols] = df[cols].replace({'Yes': 1, 'No': 0})

saved_file_path = os.path.join("data", "prepared_leads_scoring.csv")
df.to_csv(saved_file_path, index=0)

In [32]:
# Identifikasi categorical dan numerical columns
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remove `Converted` kolom
numerical_cols.remove('Converted')
numerical_cols.remove('Lead Number')
categorical_cols.remove('Prospect ID')

# Pipeline untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# Pipeline untuk fitur kategorik
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Not Specified')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# menggabungkan transformer menjadi preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Splitting the data into training and testing sets
X = df.drop(columns=['Converted', 'Prospect ID', 'Lead Number'])
y = df['Converted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Applying the preprocessor to the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Displaying the shape of the processed data
X_train_preprocessed.shape, X_test_preprocessed.shape

((7392, 123), (1848, 123))